# Homework solution. Week 5.

References:
- https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/week_5_batch_processing/setup/macos.md

### Question 1: 

**Install Spark and PySpark** 

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?
- 3.3.2 (*)
- 2.1.4
- 1.2.3
- 5.4
</br></br>

In [1]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 19.0.1
Branch HEAD
Compiled by user yumwang on 2022-10-15T09:47:01Z
Revision fbbcf9434ac070dd4ced4fb9efe32899c6db12a9
Url https://github.com/apache/spark
Type --help for more information.


### Question 2: 

**HVFHW June 2021**

Read it with Spark using the same schema as we did in the lessons.</br> 
We will use this dataset for all the remaining questions.</br>
Repartition it to 12 partitions and save it to parquet.</br>
What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.</br>


- 2MB
- 24MB (*)
- 100MB
- 250MB
</br></br>

In [37]:
from pyspark.sql import SparkSession, types
import pyspark.sql.functions as F


spark = SparkSession.builder\
    .master("local[*]")\
    .appName("de-zoomcamp")\
    .getOrCreate()

schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

# spark.stop()

#### Load data

In [17]:
data = spark.read.csv("./data/fhvhv_tripdata_2021-06.csv.gz", header=True, schema=schema)

In [21]:
data.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [18]:
data.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
+--------------------+------------------

In [30]:
data.pickup_datetime

Column<'pickup_datetime'>

In [23]:
data.repartition(12).write.mode("overwrite").parquet("./data/data.parquet")

23/03/09 21:38:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [45]:
!tree ./data

./data
├── data.parquet
│   ├── _SUCCESS
│   ├── part-00000-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00001-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00002-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00003-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00004-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00005-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00006-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00007-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00008-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00009-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   ├── part-00010-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
│   └── part-00011-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
└── fhvhv_tripdata_2021-06.csv.gz

1 directory,

Let's check our files size

In [84]:
!du -sh ./data/data.parquet/*

  0B	./data/data.parquet/_SUCCESS
 24M	./data/data.parquet/part-00000-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00001-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00002-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00003-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00004-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00005-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00006-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00007-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00008-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00009-31e1b084-9eb2-43f8-bc97-b28c7d9b10eb-c000.snappy.parquet
 24M	./data/data.parquet/part-00010-

### Question 3: 

**Count records**  

How many taxi trips were there on June 15?</br></br>
Consider only trips that started on June 15.</br>

- 308,164
- 12,856
- 452,470 (*)
- 50,982
</br></br>

In [38]:
data.select(
    F.min(col=data.pickup_datetime).alias("min_date"),
    F.max(col=data.pickup_datetime).alias("max_date")).show()

+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|2021-06-01 00:00:00|2021-06-30 23:59:59|
+-------------------+-------------------+



In [44]:
data.filter(F.to_date(data.pickup_datetime) == "2021-06-15").count()

452470

### Question 4: 

**Longest trip for each day**  

Now calculate the duration for each trip.</br>
How long was the longest trip in Hours?</br>

- 66.87 Hours (*)
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours
</br></br>

In [67]:
data.withColumn("trip_distance", (data.dropoff_datetime - data.pickup_datetime).cast("long"))\
    .withColumn("trip_distance_hours", F.col("trip_distance") / 3600)\
    .orderBy(F.col("trip_distance_hours").desc())\
        .show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------+-------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|trip_distance|trip_distance_hours|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------+-------------------+
|              B02872|2021-06-25 13:55:41|2021-06-28 08:48:25|          98|         265|      N|                B02872|       240764|   66.8788888888889|
|              B02765|2021-06-22 12:09:45|2021-06-23 13:42:44|         188|         198|      N|                B02765|        91979| 25.549722222222222|
|              B02879|2021-06-27 10:32:29|2021-06-28 06:31:20|          78|         169|      N|                B02879|        71931| 19.980833333333333|
|              B02800|2021-06-26 22:37:11|2021-06-27 16:49:01|         263| 

### Question 5: 

**User Interface**

 Spark’s User Interface which shows application's dashboard runs on which local port?</br>

- 80
- 443
- 4040 (*)
- 8080
</br></br>

![](./img/spark-app.png)

We can double-check if the app is running on the port 4040 in the terminal

In [9]:
!lsof -i :4040

COMMAND   PID     USER   FD   TYPE             DEVICE SIZE/OFF NODE NAME
java    48766 a_kulesh  312u  IPv6 0xe4db21caff62f8c9      0t0  TCP *:yo-main (LISTEN)


### Question 6: 

**Most frequent pickup location zone**

Load the zone lookup data into a temp view in Spark</br>
[Zone Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv)</br>

Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?</br>

- East Chelsea
- Astoria
- Union Sq
- Crown Heights North (*)
</br></br>

In [72]:
taxi_zone_lookup = spark.read.csv("./data/taxi_zone_lookup.csv", header=True, inferSchema=True)
taxi_zone_lookup.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [82]:
data.select(data.PULocationID.alias("LocationID"))\
    .join(taxi_zone_lookup, on="LocationID")\
        .groupBy(F.col("Zone"))\
        .agg(F.count("*").alias("count"))\
            .orderBy(F.col("count").desc()).show(5)

+-------------------+------+
|               Zone| count|
+-------------------+------+
|Crown Heights North|231279|
|       East Village|221244|
|        JFK Airport|188867|
|     Bushwick South|187929|
|      East New York|186780|
+-------------------+------+
only showing top 5 rows

